# FBiH Merge Processed Data

In [3]:
import pandas as pd

from config import SAVE_DIR

## Non political data

**Some Notes on the tables**

`gross_average_wages.xlsx` and `employees.xlsx` both have the same amount of records and go from 2005 to 2024.

`census_2013.xlsx` is only for the year 2013 which is fine as it will be supplemented with political data and be used as a sanity check for making the assumption that people vote based on ethnic lines.

`businesses_per_sector.xlsx` only goes form 2012 to 2023. It might be useful as a proxy for education given the amount of businesses in extractive sectors such as agriculture and/or mining.

Loading all the tables up

In [4]:
gaw = pd.read_excel(SAVE_DIR + "gross_average_wages.xlsx")
registar = pd.read_excel(SAVE_DIR + "businesses_per_sector.xlsx")
census = pd.read_excel(SAVE_DIR + "census_2013.xlsx")
employees = pd.read_excel(SAVE_DIR + "employees.xlsx")

I will do left joins to preserve rows that may not match up perfectly with other tables.

In [5]:
gaw_reg = pd.merge(gaw, registar, on=["Municipality", "Year"], how="left")
gaw_reg_employees = pd.merge(gaw_reg, employees, on=["Municipality", "Year"], how="left")
total = pd.merge(gaw_reg_employees, census, on=["Municipality", "Year"], how="left")

Going to save this for now

In [18]:
total.to_excel(SAVE_DIR + "merged_no_politics.xlsx", index=False)

In [6]:
df = pd.read_excel(SAVE_DIR + "merged_no_politics.xlsx")
df

,Municipality,Year,Gross Average Wage,00-Unclassified according to activities CEA 1),"A-Agriculture, forestry and fishing",Activities of households as employers; undifferentiated goods and services-producing activities of households for own use,B-Minning and quarrying,C-Manufacturing,"D-Electricity, gas, steam and air conditioning supply","E-Water supply, sewerage, waste management and remediation activities",...,O-Public administration and defence; compulsory social security,P-Education,Q-Human health and social work activities,"R-Arts, entertainment and recreation",S-Other service activities,Total,Employees,Bosniak,Croat,Serb
0,grad bihac,2005,824.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11185.0,NaN,NaN,NaN
1,bosanska krupa,2005,717.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2272.0,NaN,NaN,NaN
2,bosanski petrovac,2005,655.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1286.0,NaN,NaN,NaN
3,bu¸im,2005,677.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1105.0,NaN,NaN,NaN
4,cazin,2005,721.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4908.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,drvar,2024,1492.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1081.0,NaN,NaN,NaN
1576,glamoc,2024,1708.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,628.0,NaN,NaN,NaN
1577,kupres,2024,2080.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,979.0,NaN,NaN,NaN
1578,livno,2024,2104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4865.0,NaN,NaN,NaN


Most of the data is available from `2012` to `2023`.

## Transform and Merge Political data

Below will be use full for processing data in the db to be presentable as a df.

SELECT
    m.name AS municipality_name,
    y.year_id,
    
    SUM(cr.votes) AS total_votes,

    SUM(CASE 
        WHEN c.ethnicity = 'B' THEN cr.votes 
        ELSE 0 
    END) AS bosnian_votes,
    
    (SUM(CASE 
        WHEN c.ethnicity = 'B' THEN cr.votes 
        ELSE 0 
    END) * 100.0) / SUM(cr.votes) AS percentage_bosnian,

    SUM(CASE 
        WHEN c.ethnicity = 'C' THEN cr.votes 
        ELSE 0 
    END) AS croat_votes,
    
    (SUM(CASE 
        WHEN c.ethnicity = 'C' THEN cr.votes 
        ELSE 0 
    END) * 100.0) / SUM(cr.votes) AS percentage_croat

FROM
    Municipalities AS m
JOIN
    Years AS y ON m.year_id = y.year_id
JOIN
    CandidateResults AS cr ON m.municipality_id = cr.municipality_id
JOIN
    Candidates AS c ON cr.candidate_id = c.candidate_id
WHERE
    y.year_id = 2024
GROUP BY
    m.name, y.year_id;